# Data Preparation

In [6]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.facecolor'] = 'white' # Since I use a dark IDE

# To allow multiple outputs per cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection  import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

## Reading in Data

In [7]:
unwanted_cols = ['Unnamed: 0', 'Subject']

wba_df = pd.read_csv("wba_data_CLEAN.csv")
wba_df = wba_df.drop(columns=unwanted_cols)

wba_gen = pd.read_csv("wba_gen.csv")
wba_gen = wba_gen.drop(columns='Unnamed: 0')

In [8]:
# wba_gen = wba_gen[wba_gen['OverallPoF'] == 1]
wba_df = wba_df[wba_df['OverallPoF'] == 1]

### Feature Selection

From `feature_selection.ipynb`

In [ ]:
# # Add selected features here
# features = wba_df.columns 

# wba_df = wba_df

Limiting normative data to same columns as generated

In [9]:
wba_df = wba_df[wba_gen.columns]

# Machine Learning

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

## Train and Test Sets

We will use the generated data as training and normative data as testing

In [10]:
x_vars = [c for c in wba_df.columns if c != "OverallPoF"]

In [11]:
y_train = wba_gen['OverallPoF'].copy()
y_test = wba_df['OverallPoF'].copy()

X_train = wba_gen[x_vars].copy()
X_test = wba_df[x_vars].copy()

### Scaling

In [17]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.transform(X_test)

## Support Vector Machine

In [18]:
from sklearn import svm

# For convenient performance metrics
from sklearn.metrics import classification_report

In [54]:
svm_classifier = svm.SVC(
  kernel = "rbf",
  decision_function_shape="ovo"
  )

svm_classifier.fit(X_train_scaled, y_train)

SVC(decision_function_shape='ovo')

### Prediction and Accuracy

In [55]:
# Predictions
y_test_pred_svm = svm_classifier.predict(X_test_scaled)

In [16]:
accuracy_score(y_test, y_test_pred_svm)
# print(classification_report(y_test, y_test_pred_svm))

NameError: name 'y_test_pred_svm' is not defined

## Random Forest

In [1]:
from sklearn.ensemble import RandomForestClassifier

### GridSearch

In [4]:
param_grid = {'n_estimators': [100, 200, 300],
                'criterion': ['gini', 'entropy', 'log_loss']} 

rf_classifier = RandomForestClassifier()

grid_rf = GridSearchCV(
  rf_classifier,
  param_grid, 
  cv = 3
  )

In [12]:
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'n_estimators': [100, 200, 300]})

In [13]:
rf_classifier = grid_rf.best_estimator_

In [25]:
rf_pred = rf_classifier.predict(X_test)
accuracy_score(rf_pred, y_test)

0.7105263157894737